# Tracing Custom Dummy App with OTEL Spans using _TruLens_

This notebook demonstrates the "otel_tracing" experimental feature in _TruLens_.
This enables the collection of _OpenTelemetry_ spans during app evaluation. Data
that is collected by _TruLens_ is recorded as spans.

- OTEL exporters demonstrated in this notebook are:

  - Console exporter (prints exported spans in the console or stream).

  - In-memory exporter. This stores spans in python list you can access in this
    notebook.

  - _Zipkin_ exporter. Setup below includes `docker` commands to download and
    start a _Zipkin_ collector for demonstration purposes. To open the UI for
    this exporter, open _Docker Desktop_, click on the triple dots under
    "Actions" for the zipkin container and select "Open with browser".

In [ ]:
# Install OTEL deps:
# ! pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp

# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:which
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
# ruff: noqa: F401

from pathlib import Path
import sys

import dotenv
from opentelemetry import trace
from opentelemetry.exporter.zipkin.json import ZipkinExporter  # zipkin exporter
from opentelemetry.sdk.trace.export import (
    ConsoleSpanExporter,  # console exporter
)
from opentelemetry.sdk.trace.export.in_memory_span_exporter import (
    InMemorySpanExporter,  # in-memory exporter
)
from trulens.apps.custom import TruCustomApp
from trulens.core import Feedback
from trulens.core import Select
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.trace import sem
from trulens.experimental.otel_tracing.core.trace.trace import (
    trulens_tracer_provider,
)
from trulens.feedback.dummy.provider import DummyProvider
from trulens.semconv.trace import SpanAttributes

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))


dotenv.load_dotenv()

In [ ]:
# Sets the global default tracer provider to be the trulens one.
trace.set_tracer_provider(trulens_tracer_provider())

# Creates a tracer for custom spans below.
tracer = trace.get_tracer(__name__)

In [ ]:
# Setup in-memory span exporter.
exporter = InMemorySpanExporter()

# Setup console/file/string exporter
# stream = StringIO()

# Will print A LOT to stdout unless we set a different stream.
# exporter = ConsoleSpanExporter(out=stream)

# Setup zipkin exporter
# exporter = ZipkinExporter(endpoint="http://localhost:9411/api/v2/spans")

# Create a TruLens session.
session = TruSession()

session.reset_database()  # reset the database first as enabling otel_tracing creates another table which we don't want to be deleted by reset_database.

# To export spans to an external OTEL SpanExporter tool, set it here:
session.experimental_otel_exporter = exporter

# (Optional) Enable otel_tracing. Note that this is not required if you set the
# exporter above. If you would like to trace using spans without an exporter,
# this step is required.
session.experimental_enable_feature("otel_tracing")

# session.start_dashboard()

In [ ]:
from examples.dev.dummy_app.app import DummyApp

# Create dummy endpoint for a dummy feedback function:
dummy_provider = DummyProvider()
dummy_feedback = Feedback(dummy_provider.sentiment).on(
    text=Select.RecordSpans.trulens.call.generate.bound_arguments.prompt
)
# Parts of the selector are:
#
# - Select.RecordSpans - Select spans dictionary, organized by span name.
#
# - trulens.call.generate - Span name. TruLens call spans are of the form
#   "trulens.call.<method_name>".
#
# - attributes - Span attributes
#
# - ["trulens.bound_arguments"] - Attributes specific to TruLens spans. Call spans
#   include method call arguments in "trulens.bound_arguments". Other attributes are
#   "trulens.ret" for the call span's return value and "trulens.error" for the
#   call span's error if it raised an exception instead of returning.
#
# - prompt - The prompt argument to the method call named.

# Create custom app:
ca = DummyApp()

# Create trulens wrapper:
ta1 = TruCustomApp(
    ca,
    app_name="customapp1",
    feedbacks=[dummy_feedback],
)

ta2 = TruCustomApp(
    ca,
    app_name="customapp2",
    #    feedbacks=[dummy_feedback],
)

In [ ]:
# Normal trulens recording context manager:

# print(trace.get_current_span())

with ta1 as recorder:
    # (optional) Custom span.
    with tracer.start_as_current_span("custom inner span") as inner_span:
        print("inner_span=", inner_span)

        # (optional) Set custom span attributes.
        inner_span.set_attribute("custom", "value1")

        # Normal instrumented call:
        print(ca.respond_to_query(query="hello"))

        with ta2 as recorder2:
            # Second custom span.
            with tracer.start_as_current_span(
                "custom inner span 2"
            ) as inner_span2:
                print("inner_span2=", inner_span)
                inner_span2.set_attribute("custom", "value2")

                # Second instrumented call.
                print(ca.respond_to_query(query="hi"))

            # A third instrumented call.
            print(ca.respond_to_query(query="goodbye"))

record, *rest = recorder.records

In [ ]:
# Check the feedback results. Note that this feedback function is from a dummy
# provider which does not have true sentiment analysis.

print(record.feedback_results[0].result())

In [ ]:
record.experimental_otel_spans[1]

In [ ]:
record.experimental_otel_spans

In [ ]:
record.experimental_otel_spans[0].attributes

In [ ]:
# Check trulens instrumented calls as spans:

span = record.get(Select.RecordSpans.trulens.call.respond_to_query[0])

In [ ]:
# Raw representation of attributes in the span:

span._attributes

In [ ]:
# Serialized versions of attributes:

span.attributes

In [ ]:
# Some of the same data is also accessible directly from the span instance:

# These are the explicitly defined pydantic model fields.
for key, value in span.model_fields.items():
    print(f"(explicit) {key}: {getattr(span, key)}")

# These are computed properties defined to mirror their value in the attribute dict:
for key, value in span.model_computed_fields.items():
    print(f"(computed) {key}: {getattr(span, key)}")

In [ ]:
# Span type can be accessed by either the span_types attributes of enums or
# python class hierarchy:

print(span.span_types)
print(type(span).__bases__)

In [ ]:
# Use span_types for checking span category:

print(SpanAttributes.SpanType.CALL in span.span_types)
print(SpanAttributes.SpanType.UNKNOWN in span.span_types)

In [ ]:
# Use isinstance instead:

print(isinstance(span, sem.Call))
print(isinstance(span, sem.Unknown))

In [ ]:
# All of the spans listed above should be visible in the chosen exporter.

# The InMemorySpanExporter stores the spans in memory. Lets read them back here
# to inspect them:

from trulens.experimental.otel_tracing.core.trace import context as core_context
from trulens.experimental.otel_tracing.core.trace import otel as core_otel

if "exporter" in locals():
    print(f"Spans exported to {exporter}:")

    if isinstance(exporter, InMemorySpanExporter):
        spans = exporter.get_finished_spans()

        for span in spans:
            # Using of_contextlike here to print ids more readably.
            print(
                core_context.SpanContext.of_contextlike(span.context),
                "->",
                core_context.SpanContext.of_contextlike(span.parent),
                span.name,
            )

    elif isinstance(exporter, ZipkinExporter):
        print(
            "The spans should be visible in the zipkin dashboard at http://localhost:9411/zipkin/"
        )

In [ ]:
# Get the spans back from the new spans table.

db = session.connector.db

for span in db.get_spans():
    print(span)
    if isinstance(span, sem.RecordRoot):
        print("\tThis is the root of record:", span.record_id)
        print("\tRecord is for app:", span.app_id)
    if isinstance(span, sem.Record):
        print("\tBelongs to records:", list(span.record_ids.values()))

In [ ]:
# Get the trace roots only:

from trulens.semconv.trace import SpanAttributes

db = session.connector.db

for span in db.get_spans(
    where=db.Span.span_types.contains(SpanAttributes.SpanType.RECORD_ROOT)
):
    # Root spans identify the record_id they are recording and the app to which
    # that record belongs:
    print(span, span.record_id, span.app_id, span.app_name, span.app_version)

    # Root spans have these fields that were previously in Record:
    print("\tmain_input:", repr(span.main_input))
    print("\tmain_output:", repr(span.main_output))
    print("\tmain_error:", repr(span.main_error))
    print("\ttotal_cost:", span.total_cost)

    # Get the spans for a specific record:
    for child_span in db.get_spans(
        where=db.Span.record_ids.contains(span.record_id)
    ):
        print("\t", child_span)